In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

#year = 2022
name = 'KCE'
buy_date = '2021-10-07'

In [2]:
format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','profit':'{:,.2f}',
              'pct':'{:,.2f}%','net':'{:,.2f}',
              'cost_amt':'{:,.2f}','unit_cost':'{:,.2f}','avg_cost':'{:,.2f}'}

pd.set_option('display.float_format','{:,.2f}'.format)

### Record selection for transactions

In [3]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = '%s' AND buys.date >= '%s'
ORDER BY sell_year DESC, sell_month DESC, buy_month DESC,name'''

sql = sql % (name, buy_date)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = 'KCE' AND buys.date >= '2021-10-07'
ORDER BY sell_year DESC, sell_month DESC, buy_month DESC,name


In [5]:
sells_df

,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,profit,kind
0,KCE,2021,12,2021,12,91.25,87.00,4.25,1000,"91,250.00","87,000.00","4,250.00",4.89,"3,855.19",DTD
1,KCE,2021,12,2021,10,93.00,80.00,13.00,1000,"93,000.00","80,000.00","13,000.00",16.25,"12,616.82",DTD
2,KCE,2021,11,2021,10,91.00,80.00,11.00,1000,"91,000.00","80,000.00","11,000.00",13.75,"10,621.25",DTD


In [6]:
sells_df[['gross','profit','qty']].sum()

gross    28,250.00
profit   27,093.26
qty       3,000.00
dtype: float64

In [7]:
profit = 27093.26
profit

27093.26

### Record selection for active stocks

In [8]:
sql = '''
SELECT name, buys.date AS buy_date,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt, net
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = '%s' AND status = 'Active'
ORDER BY name, buys.date DESC'''
sql = sql % name
buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)

,name,buy_date,unit_cost,qty,cost_amt,net
0,KCE,2022-03-03,58.00,"1,000","58,000.00","58,128.46"
1,KCE,2022-02-11,61.50,"1,000","61,500.00","61,636.22"
2,KCE,2022-02-09,67.00,"3,000","201,000.00","201,445.19"
3,KCE,2022-01-18,81.00,"2,000","162,000.00","162,358.81"
4,KCE,2021-12-14,87.00,"2,000","174,000.00","174,385.39"
5,KCE,2021-10-07,87.00,"2,000","174,000.00","174,385.39"


In [9]:
buys_df[['cost_amt','net','qty']].sum()

cost_amt   830,500.00
net        832,339.46
qty         11,000.00
dtype: float64

In [10]:
sql = '''
SELECT name, p_date, FORMAT(amt,2) AS amt, FORMAT(net,2) AS net, format(amt-net,2) AS refund,
year, quarter AS q, x_date, number, ppu, (net / number * 1000) AS div_1k
FROM dividends
WHERE name = '%s' AND x_date > '%s'
ORDER BY p_date DESC'''
sql = sql % (name, buy_date)
dtl_by_month = pd.read_sql(sql, conpf)
dtl_by_month

,name,p_date,amt,net,refund,year,q,x_date,number,ppu,div_1k


In [11]:
new_ppu = 0.4
new_qty = 11_000
new_div = new_qty * new_ppu * .96
new_div

4224.0

In [12]:
net_profit = profit + new_div
net_profit

31317.26

### Plan A: Sell 7,000 shares at 64.75